In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display

from deep_orderbook.learn.test_learn import train_and_predict
from deep_orderbook.visu import Visualizer
from deep_orderbook.config import TrainConfig, ReplayConfig, ShaperConfig


vis = Visualizer()

train_config = TrainConfig()
replay_config = ReplayConfig()
shaper_config = ShaperConfig()

# Define your asynchronous function to update the figure
async def online_learning(train_config, replay_config, shaper_config):
    async for shaped, t2l, pxar, prediction, loss in train_and_predict(config=train_config, replay_config=replay_config, shaper_config=shaper_config):
        # Process your data for heatmaps
        im_data = shaped.copy().transpose(1, 0, 2)
        im_data[:, :, 0] *= -0.5
        im_data[:, :, 1:3] *= 1e6
        im_data = np.clip(im_data, -1, 1)
        im_data = im_data.mean(axis=2)
        t2l_data = np.clip(t2l[:, :, 0].T, -1, 1)

        pred_shape = prediction.reshape(t2l.shape).transpose(1, 0, 2)
        pred_shape = np.clip(pred_shape[:, :, 0], -1, 1)

        vis.add_loss(loss)
        vis.update(books_z_data=im_data, level_reach_z_data=t2l_data, bidask=pxar, pred_shape=pred_shape)

await online_learning(train_config=train_config, replay_config=replay_config, shaper_config=shaper_config)

FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(103,0,31)'], [0.1, 'rgb(178,24,43)'],
                             [0.2, 'rgb(214,96,77)'], [0.3, 'rgb(244,165,130)'],
                             [0.4, 'rgb(253,219,199)'], [0.5, 'rgb(247,247,247)'],
                             [0.6, 'rgb(209,229,240)'], [0.7, 'rgb(146,197,222)'],
                             [0.8, 'rgb(67,147,195)'], [0.9, 'rgb(33,102,172)'],
                             [1.0, 'rgb(5,48,97)']],
              'showscale': False,
              'type': 'heatmap',
              'uid': '4cf8b5b6-f0d0-4f57-ac80-fbbb83b6e56d',
              'xaxis': 'x',
              'yaxis': 'y',
              'z': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
                          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
  

device=device(type='cuda')


  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]